In [ ]:
%matplotlib tk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plot_interactions import on_hover, on_click


In [ ]:
# Check which Python interpreter and kernel you're using
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")


In [ ]:
basename = "slice_pa_data"
slice_num = 10  # Which slice to analyze


In [ ]:
dfs = [pd.DataFrame() for _ in range(4)]

# Load the trigger file to determine number of slices
trig_df = pd.read_csv(f"data/{basename}_trig.csv", sep=r"\s+", header=None)
num_slices = len(trig_df)

# Load the files
for i in range(4):
  dfs[i] = pd.read_csv(f"data/{basename}_bd_{i}.csv", sep=r"\s+", header=None)

# Initialize the array
adc_array = dfs[0].to_numpy()

# Append each subsequent DataFrame to the array (in the second dimension)
for i in range(1, 4):
  adc_array = np.concatenate((adc_array, dfs[i].to_numpy()), axis=1)

# Convert ADC values to amps (-2^15 to 2^15 -> -5.0 to 5.0 A)
adc_array = adc_array * (5.0 / 32768)

# Calculate slice_size based on total samples and number of slices
total_samples = adc_array.shape[0]
slice_size = total_samples // num_slices
num_channels = adc_array.shape[1]

print(f"Detected {num_slices} slices from trigger file")
print(f"Total samples: {total_samples}, Slice size: {slice_size}")
print(f"Number of channels: {num_channels}")

time = np.arange(1, slice_size + 1) * 20e-6



In [ ]:
# Perform FFT analysis on specified slice
if slice_num >= num_slices:
    print(f"Error: slice_num {slice_num} is out of range (max: {num_slices-1})")
else:
    # Select data for FFT (skip first 20 samples)
    slice_data = adc_array[(slice_num*slice_size + 20):(slice_num+1)*slice_size, :]
    fft_title = f'Frequency Domain - All Channels (Slice {slice_num}) - Hover to identify'
    fft_title_max = f'Frequency Domain - Max Over Channels (Slice {slice_num})'
    
    # Compute FFT for all columns
    fft_results = []
    fft_magnitudes = []
    
    for col in range(slice_data.shape[1]):
      fft_result = np.fft.fft(slice_data[:, col])
      fft_magnitude = np.abs(fft_result)
      fft_results.append(fft_result)
      fft_magnitudes.append(fft_magnitude)
    
    # Convert to array for easier manipulation
    fft_magnitudes = np.array(fft_magnitudes).T
    
    # Compute frequencies
    fft_frequencies = np.fft.fftfreq(slice_data.shape[0], time[1] - time[0])
    positive_freq_mask = fft_frequencies > 0
    
    # Create two subfigures
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # First subplot: All columns with hover interaction
    fft_lines = []
    for col in range(slice_data.shape[1]):
      line, = ax1.plot(fft_frequencies[positive_freq_mask] / 1000, fft_magnitudes[positive_freq_mask, col], 
                       alpha=0.5, linewidth=2)
      fft_lines.append(line)
    
    ax1.set_xlabel('Frequency (kHz)')
    ax1.set_ylabel('Magnitude')
    ax1.set_title(fft_title)
    ax1.set_ylim(0, 10000)
    ax1.grid(True)
    
    # Create annotation for FFT plot hover
    fft_annotation = ax1.annotate('', xy=(0,0), xytext=(20,20), textcoords='offset points',
                                  bbox=dict(boxstyle='round', fc='yellow', alpha=0.9),
                                  arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'),
                                  visible=False, fontsize=12, weight='bold')
    
    # Store currently highlighted line for FFT plot
    fft_highlighted_line = [None]
    
    # Store original y-limits for reset
    fft_original_ylim = ax1.get_ylim()
    
    # Connect hover and click events to first subplot
    fig.canvas.mpl_connect('motion_notify_event', 
                           lambda event: on_hover(event, ax1, fft_lines, fft_annotation, fft_highlighted_line))
    fig.canvas.mpl_connect('button_press_event', 
                           lambda event: on_click(event, ax1, fft_lines, fft_original_ylim))
    
    # Second subplot: Max over channels
    max_magnitude = np.max(fft_magnitudes, axis=1)
    ax2.plot(fft_frequencies[positive_freq_mask] / 1000, max_magnitude[positive_freq_mask])
    ax2.set_xlabel('Frequency (kHz)')
    ax2.set_ylabel('Magnitude')
    ax2.set_title(fft_title_max)
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
